In [1]:
import dash
import alpha_vantage.timeseries as av
import pandas as pd
import os
from dash import html, dcc, Input, Output
from dotenv import load_dotenv

load_dotenv()
api_key = os.environ.get('API_KEY')

external_stylesheets = ['styles.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [2]:
app.layout = html.Div([
    html.H1("Stock Data Dashboard"),
    
    html.Label("Enter Stock Symbol:"),
    dcc.Input(id='stock-input', type='text', value='AAPL'),
    
    html.Button(id='submit-button', n_clicks=0, children='Submit'),
    
    dcc.Graph(id='stock-chart', figure={}),
])

In [3]:
@app.callback(
    Output('stock-chart', 'figure'),
    Input('submit-button', 'n_clicks'),
    [dash.dependencies.State('stock-input', 'value')]
)

def update_chart(n_clicks, stock_symbol):
    # Making API request to Alpha Vantage
    ts = av.TimeSeries(key=api_key, output_format="pandas")
    data, meta_data = ts.get_weekly(symbol=stock_symbol)

    # Processing the data
    df = data.rename(
        columns={
            "1. open": "Open",
            "2. high": "High",
            "3. low": "Low",
            "4. close": "Close",
            "5. volume": "Volume",
        }
    )

    # Creating a candlestick chart
    figure = {
        "data": [
            {
                "x": df.index,
                "open": df["Open"],
                "high": df["High"],
                "low": df["Low"],
                "close": df["Close"],
                "type": "candlestick",
                "name": stock_symbol,
            },
        ],
        "layout": {
            "title": f"{stock_symbol} Stock Price",
            "xaxis": {"rangeslider": {"visible": False}},
            "yaxis": {"title": "Stock Price"},
        },
    }

    return figure

In [4]:
if __name__ == '__main__':
    app.run_server(debug=True)